# Generate Fix Commit Tree

If the same (project, file, line) sequence has a fix-commit
that is appeared as fix-commit-parent for the same sequence,
we are saying the first fix created / left a bug.

This notebook will find such commit sequence and make a tree from those sequence.
This will help to investigate in which situation
developers create / leave a bug while fixing another.

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_json('../dataset/sstubsLarge')

In [3]:
print('Rows: {}, Columns: {}'.format(*df.shape))

Rows: 63923, Columns: 14


In [4]:
print('**Columns:**')
print(*df.columns, sep='\n')

**Columns:**
bugType
fixCommitSHA1
fixCommitParentSHA1
bugFilePath
fixPatch
projectName
bugLineNum
bugNodeStartChar
bugNodeLength
fixLineNum
fixNodeStartChar
fixNodeLength
sourceBeforeFix
sourceAfterFix


## Filter tree-commits

First, the fix-commits appearing as parent-commit in another entry are found via SQL.
Three different SQL queries are run producing the same result to grow confidence.
Then this query is applied to `pd.DataFrame`.

In [5]:
import sqlite3
conn = sqlite3.connect('../database/sstubs.db')
cursor = conn.cursor()

In [6]:
query = '''SELECT count(*)
    FROM (SELECT * 
    FROM sstubs_large AS b1
        INNER JOIN sstubs_large AS b2
        ON b1.parent = b2.child AND b1.project = b2.project AND b1.file = b2.file AND b1.line = b2.line
    GROUP BY b1.child, b1.parent, b1.project, b1.line, b1.file, b1.type)'''
for res in cursor.execute(query):
    print(*res)

400


In [7]:
query = '''SELECT count(*) FROM sstubs_large WHERE (parent, project, file, line) IN (
                SELECT child, project, file, line FROM sstubs_large
           ) '''
for row in cursor.execute(query):
    print(*row)


400


In [8]:
query = '''SELECT count(*) FROM sstubs_large AS P WHERE EXISTS (
                SELECT parent
                FROM sstubs_large AS C
                WHERE C.child = P.parent AND C.project = P.project AND C.file = P.file AND C.line = P.line
           ) '''
for row in cursor.execute(query):
    print(*row)

400


In [25]:
columns = [
    'fixCommitSHA1',
    'bugFilePath',
    'projectName',
    'bugLineNum',
    'bugType',
    'sourceBeforeFix',
    'sourceAfterFix',
]
child_df = df[['fixCommitParentSHA1', *columns]]
parent_df = df[columns]\
    .rename(columns={'fixCommitSHA1': 'fixCommitParentSHA1'})
rename_cols = {
    'sourceBeforeFix_y': 'sourceA', # initial source in parent
    # later source in parent === initial source in child (sourceBeforeFix_x)
    'sourceAfterFix_y': 'sourceB',
    'sourceAfterFix_x': 'sourceC', # later source in child
    'bugType_y': 'bugTypeA', # parent bug type
    'bugType_x': 'bugTypeB' # child bug type
}

merged_df = child_df.merge(
    parent_df,
    how='inner',
    on=['fixCommitParentSHA1', 'bugFilePath', 'projectName', 'bugLineNum'],
).drop(columns='sourceBeforeFix_x').rename(columns=rename_cols)

In [35]:
merged_df[merged_df.sourceA.str.len() < 40].sample(5)[[
    'sourceA',
    'bugTypeA',
    'sourceB',
    'bugTypeB',
    'sourceC',
]]

,sourceA,bugTypeA,sourceB,bugTypeB,sourceC
355,countNewlines(headerFragment) + 1,CHANGE_NUMERAL,countNewlines(headerFragment) + 2,CHANGE_NUMERAL,countNewlines(headerFragment) + 1
83,"ref_val.startsWith(""HTTP error 403"")",LESS_SPECIFIC_IF,"ref_val.startsWith(""HTTP error 403"") || ref_va...",DIFFERENT_METHOD_SAME_ARGS,"ref_val.contains(""HTTP error 403"")"
395,ret.setComplete(response),CHANGE_IDENTIFIER,"ret.setComplete(e,response)",CHANGE_IDENTIFIER,ret.setComplete(response)
310,new Paint(),OVERLOAD_METHOD_MORE_ARGS,new Paint(Paint.ANTI_ALIAS_FLAG),OVERLOAD_METHOD_DELETED_ARGS,new Paint()
370,"postfix.equals(""id"")",DIFFERENT_METHOD_SAME_ARGS,"postfix.startsWith(""id"")",CHANGE_IDENTIFIER,"postfix.equals(""id"")"


In [36]:
columns = [
    'fixCommitParentSHA1',
    'fixCommitSHA1',
    'projectName',
    'bugFilePath',
    'bugLineNum',
    'bugTypeA',
    'bugTypeB',
    'sourceA',
    'sourceB',
    'sourceC',
]
tree_df = pd.DataFrame(
    data=merged_df.groupby(columns).groups.keys(),
    columns=columns,
)
tree_df.shape

(562, 10)

In [37]:
tree_df.to_csv('../dataset/sequential_sstubs.csv', index=False)

### Level 3 Tree

## Build Tree

In [12]:
from collections import deque

In [13]:
roots = {}
for _, row in tree_df.iterrows():
    parent = row.fixCommitParentSHA1
    child = row.fixCommitSHA1
    project = row.projectName
    file = row.bugFilePath
    line = row.bugLineNum
    bug = row.bugType
    parentKey = (parent, project, file, line)
    if parent not in roots:
        roots[parentKey] = set()
    roots[parentKey].add((child, bug))
    childKey = (child, project, file, line)
    if childKey not in roots:
        roots[childKey] = set()
    roots[childKey].add((parent, bug))

In [14]:
len(roots)

614